In [ ]:
import marimo as mo

# Комбинирование нескольких MCP-серверов

FastMCP позволяет объединять несколько независимых серверов в один через метод `import_server()`.

Это позволяет строить модульные системы, где каждый сервер отвечает за свою область, но все они доступны через единую точку входа.

## Шаг 1: Импорты и настройка

In [ ]:
import asyncio
import os
from pathlib import Path

from dotenv import load_dotenv
from fastmcp import Client
from langchain.agents import create_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_openai import ChatOpenAI

load_dotenv()

BASE_MODEL = os.getenv("BASE_MODEL") or ""

## Шаг 2: Структура комбинированного сервера

У нас есть три файла:

- `math_server.py` - инструменты для математики (calculate, factorial)
- `text_server.py` - инструменты для текста (count_words, reverse_text, to_uppercase)
- `server_combined.py` - главный сервер, который импортирует оба через `mcp.import_server()`

Каждый сервер независим и может использоваться отдельно, но в комбинированном они работают вместе.

## Шаг 3: Как работает import_server()

В `server_combined.py` есть функция setup():

```python
async def setup():
    await mcp.import_server(math_server)
    await mcp.import_server(text_server)
```

Эта функция берет все инструменты из импортированных серверов и переносит их в главный сервер.

## Шаг 4: Подключение и использование

Для агента все инструменты выглядят как единый набор возможностей.

In [ ]:
async def main():
    server_script = Path("server_combined.py")

    async with Client(str(server_script)) as client:
        print("Available tools from combined server:")
        tools_list = await client.list_tools()
        for tool in tools_list:
            print(f"  - {tool.name}")

        tools = await load_mcp_tools(client.session)

        llm = ChatOpenAI(model=BASE_MODEL, temperature=0)

        agent = create_agent(llm, tools)

        test_pdf = Path("../../../data/test.pdf")

        response = await agent.ainvoke(
            {
                "messages": [
                    (
                        "user",
                        f"""Perform a multi-step analysis:
1. Get statistics about the document at {test_pdf}
2. Calculate the factorial of the word count
3. Reverse the document name

Please execute all these tasks and present the results clearly.""",
                    )
                ]
            }
        )

        print("\nAgent response:")
        for message in response["messages"]:
            if (
                message.type == "ai"
                and hasattr(message, "content")
                and message.content
            ):
                print(f"\n{message.content}")

## Шаг 5: Запуск

In [ ]:
asyncio.run(main())

## Преимущества модульной архитектуры

- **Переиспользуемость**: базовые серверы можно использовать в разных проектах
- **Тестируемость**: каждый сервер тестируется отдельно
- **Поддерживаемость**: изменения в одном сервере не затрагивают другие
- **Гибкость**: можно комбинировать серверы в разных конфигурациях

Такой подход позволяет строить библиотеку переиспользуемых компонентов.